# Assignment #2: Language models

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment
* Optionally read a short article on the importance of corpora


## Submission

Once you have written all the missing code and run all the cells, you will submit your notebook to an automatic marking system. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

Some imports you may need. Add others as needed.

In [1]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [2]:
# You may have to adjust the path
corpus = open('corpus/Selma.txt', encoding='utf8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [3]:
pattern = 'Nils Holgersson'
width = 25

In [4]:
# spaces match tabs and newlines
pattern = re.sub(' ', '\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub('\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [5]:
def tokenize(text,lower = False):
    if lower:
        words = re.findall('\p{L}+', text.lower())
        return words
        
    words = re.findall('\p{L}+', text)
    return words

In [6]:
words = tokenize(corpus)
words[:10]

['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [7]:
# Write your code here
uniq = len(set(words))
uniq

44256

Lowercased text

In [8]:
# Write your code here
uniqLow = len(set(tokenize(corpus,True)))
uniqLow

41032

### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [9]:
# Write your code
nonletter = '[^\.;:\?!\p{L}]+'

Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [10]:
# Write your code here
def clean(text):
    return(re.sub(nonletter, " ", text))

In [11]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [12]:
test_para = clean(test_para)
test_para

'En gång hade de på Mårbacka en barnpiga som hette Back Kajsa. Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.'

#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces.

In [13]:
# Write your code here
sentence_boundaries = '[\.;:\?!]\p{IsWhite_Space}+(\p{Lu})'


##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [14]:
# Write your code here
sentence_markup = ' </s>\n<s> \g<1>'

##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [15]:
# Write your code here
text = re.sub(sentence_boundaries, sentence_markup, test_para)

In [16]:
print(text)

En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.


The output should look like this:

`En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden.`

Insert markup codes in the beginning and end of the text

In [17]:
# Write your code here
text = "<s> " + text + " </s>"
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden. </s>


The output should look like this:

`<s> En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden. </s>`

Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex.

In [18]:
# Write your code here
reg_remove_whiteSpace_n_dot = '[\.;:\?!]|(\p{IsWhite_Space})\p{IsWhite_Space}+'
text = re.sub(reg_remove_whiteSpace_n_dot,'\g<1>',text)
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


The output should look like this:
    
`<s> En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa </s>
<s> Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden </s>`

Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [19]:
# Write your code here

def replacer(match):
    return ('<s>' + match.group(1).lower() + '</s>')

def segment_sentences(text,segment = True):
    if segment:
        text = re.sub(sentence_boundaries, sentence_markup, text)
        text = "<s> " + text + " </s>"
        text = re.sub(reg_remove_whiteSpace_n_dot,'\g<1>', text)
    text = re.sub('<s>(.*)</s>', replacer, text)
    
        
    return(text)   

In [20]:
print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s>
<s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>


Estimate roughly the accuracy of your program.

#### Tokenizing the corpus

Clean and segment the corpus

In [21]:
# Write your code here
corpus = segment_sentences(clean(corpus))

In [22]:
print(corpus[-557:])

<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s>
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s>


The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this:

```
<s> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s> 
<s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s>
<s> prästens ord tycktes redan ha gått i uppfyllelse </s>
<s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s>
<s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s>
```

You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [23]:
# Write your code here
words = re.split('[ \n\r]', corpus)

In [24]:
print(words[-101:])

['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']


The five last lines of the corpus should like this:

`['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']`



### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [25]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency

We compute the frequencies.

In [26]:
frequency = unigrams(words)
list(frequency.items())[:20]

[('<s>', 59047),
 ('selma', 52),
 ('lagerlöf', 270),
 ('nils', 87),
 ('holgerssons', 6),
 ('underbara', 23),
 ('resa', 317),
 ('genom', 688),
 ('sverige', 56),
 ('</s>', 59047),
 ('första', 525),
 ('bandet', 6),
 ('bokutgåva', 11),
 ('albert', 15),
 ('bonniers', 11),
 ('förlag', 11),
 ('stockholm', 77),
 ('den', 11624),
 ('kristliga', 2),
 ('dagvisan', 2)]

#### Bigrams

In [27]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams

In [239]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]

[(('<s>', 'selma'), 8),
 (('selma', 'lagerlöf'), 11),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 5),
 (('sverige', '</s>'), 17),
 (('</s>', '<s>'), 59046),
 (('<s>', 'första'), 11),
 (('första', 'bandet'), 1),
 (('bandet', 'bokutgåva'), 2),
 (('bokutgåva', 'albert'), 11),
 (('albert', 'bonniers'), 11),
 (('bonniers', 'förlag'), 11),
 (('förlag', 'stockholm'), 10),
 (('stockholm', '</s>'), 24),
 (('<s>', 'den'), 1375),
 (('den', 'kristliga'), 2)]

In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus. This topic will be discussed during the lab session.

### Computing the likelihood of a sentence

#### Unigrams

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_. 

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [29]:
# Write your code
amout_of_words = len(tokenize(corpus))

def unigram_lm(frequency, words):
    tablue = "=====================================================\n"
    tablue = tablue + "wi      C(wi)      #words      P(wi)\n"
    tablue = tablue + "=====================================================\n"
    insert_str = "{}{}      {}      {}\n"
    spaces = "                   "
    prob_uni = 1
    
    for word in words:
        word_length = len(word)
        space = 8 - word_length
        
        insert_str = insert_str[0:2] + spaces[0:space] + insert_str[2:]
        
        C = frequency[word]
        P = C/amout_of_words
        prob_uni = P*prob_uni
        tablue = tablue + insert_str.format(word, C, amout_of_words, P)
        insert_str = "{}{}      {}      {}\n"
    
    tablue = tablue + "=====================================================\n"
    sentence_length = len(words)
    
    geo_mean = prob_uni ** (1 / sentence_length)
    entropy_rate = -(1 / sentence_length) * math.log(prob_uni, 2)
    perplexity = 2 ** entropy_rate
    
    tablue = tablue + "Prob. unigrams:          {}".format(prob_uni) + '\n'
    tablue = tablue + "Geometric mean prob.:    {}".format(geo_mean) + '\n'
    tablue = tablue + "Entropy rate:            {}".format(entropy_rate) + '\n'
    tablue = tablue + "Perplexity:              {}".format(perplexity)
    
    print(tablue)
    
    return(perplexity)  

In [97]:
sentence = 'det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
sent_words

['det', 'var', 'en', 'gång', 'en', 'katt', 'som', 'hette', 'nils', '</s>']

In [31]:
perplexity_unigrams = unigram_lm(frequency, sent_words)

wi      C(wi)      #words      P(wi)
det     21108      1041560      0.020265755213333847
var     12090      1041560      0.011607588617074388
en      13514      1041560      0.01297476861630631
gång    1332      1041560      0.001278850954337724
en      13514      1041560      0.01297476861630631
katt    16      1041560      1.5361573025077767e-05
som     16288      1041560      0.015638081339529167
hette   97      1041560      9.312953646453396e-05
nils    87      1041560      8.352855332386037e-05
</s>    59047      1041560      0.056690925150735434
Prob. unigrams:          5.3651155337425844e-27
Geometric mean prob.:    0.0023602494649885993
Entropy rate:            8.726844932328587
Perplexity:              423.68402782577465


In [32]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

423

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [118]:
# Write your code

amout_of_words = len(tokenize(corpus))

def bigram_lm(frequency, frequency_bigrams, words):
    li = ['<s>']
    
    for word in words:
        li.append(word)
    
    tablue = "=====================================================\n"
    tablue = tablue + "wi      wi+1       Ci,i+1      C(i)      P(wi+1|wi)\n"
    tablue = tablue + "=====================================================\n"
    insert_str = "{}      {}      {}      {}      {}"
    prob_bi = 1
    
    for index in range( len(li) - 1):
        word = li[index]
        word_plus = li[index + 1]
        
        C = frequency[word]
        
        if (word, word_plus) in frequency_bigrams:
            C_plus = frequency_bigrams[(word, word_plus)] 
            P = C_plus/C    
            prob_bi = prob_bi * P
            tablue = tablue + insert_str.format(word, word_plus, C_plus, C, P) + "\n"
        else:
            C_plus = 0
            P = 0
            p = frequency[word_plus] / amout_of_words
            prob_bi = prob_bi * p
            
            insert_str = insert_str + "      *backoff:      {}\n"
            tablue = tablue + insert_str.format(word, word_plus, C_plus, C, P, p)
        insert_str = "{}      {}      {}      {}      {}"
      
    
    tablue = tablue + "=====================================================\n"
    
    sentence_length = len(words)
    
    geo_mean = prob_bi ** (1 / sentence_length)
    entropy_rate = -(1 / sentence_length) * math.log(prob_bi, 2)
    perplexity = 2 ** entropy_rate
    
    tablue = tablue + "Prob. bigrams:           {}".format(prob_bi) + '\n'
    tablue = tablue + "Geometric mean prob.:    {}".format(geo_mean) + '\n'
    tablue = tablue + "Entropy rate:            {}".format(entropy_rate) + '\n'
    tablue = tablue + "Perplexity:              {}".format(perplexity)
    
    print(tablue)
    
    return(perplexity)  

In [119]:
perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)

wi      wi+1       Ci,i+1      C(i)      P(wi+1|wi)
<s>      det      5672      59047      0.09605907158704083
det      var      3839      21108      0.1818741709304529
var      en      712      12090      0.058891645988420185
en      gång      706      13514      0.052242119283705785
gång      en      20      1332      0.015015015015015015
en      katt      6      13514      0.0004439840165754033
katt      som      2      16      0.125
som      hette      45      16288      0.002762770137524558
hette      nils      0      97      0      *backoff:      8.352855332386037e-05
nils      </s>      2      87      0.022988505747126436
Prob. bigrams:           2.376169768780815e-19
Geometric mean prob.:    0.013727382866049192
Entropy rate:            6.186799588766882
Perplexity:              72.84709764111103


In [120]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

72

In addition to this sentence, _Det var en gång en katt som hette Nils_, write five other sentences that will form your test set and run your programs on them. You will insert them in your report.

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program it following the model of bigrams.

In [254]:
# Write your code
def trigrams(words):
    trigrams = []
    
    for i in range(len(words) - 2):
        trigrams.append((words[i], words[i + 1], words[i + 2]))
        
    frequency_trigrams = {}
    
    for i in range(len(words) - 2):
        
        if trigrams[i] in frequency_trigrams:
            frequency_trigrams[trigrams[i]] += 1
        else:            
            frequency_trigrams[trigrams[i]] = 1
            
    return frequency_trigrams

In [255]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

330

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [256]:
starting_text = 'De'.lower()

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies.

In [257]:
cand_nbr = 5

In [258]:
# Write your code here
def suggestion_words_bi(word):
    suggestion_dic = {}
    sentence_start = '<s>'
    word_length = len(word)
    
    items = frequency_bigrams.items()
    
    for item in items:
        key = item[0]
        start_word = key[0]
        follow_word = key[1]

        if sentence_start == start_word:           
            if word == follow_word[0:word_length]:
                suggestion_dic[follow_word] = item[1]
    
    suggestion_list_temp = (sorted(suggestion_dic.items(), key=lambda item: item[1]))
    length = len(suggestion_list_temp)
    
    suggestion_list_temp = suggestion_list_temp[length - cand_nbr:length]
    
    suggestion_list = []
    
    for tupel in suggestion_list_temp:
        suggestion_list.insert(0, tupel[0])

    return(suggestion_list)              
            
current_word_predictions_1 = suggestion_words_bi(starting_text)      

In [259]:
current_word_predictions_1

['det', 'de', 'den', 'detta', 'denna']

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [338]:
current_text = "Det var en ".lower()

Tokenize this text and return a list of tokens. Call it `tokens`.

In [339]:
# Write your code here
tokens = tokenize(current_text)

In [340]:
tokens

['det', 'var', 'en']

Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`

In [357]:
# Write your code here
frequency_trigrams = trigrams(words)

def suggestion_words_tri(word, typing_word = ''):
    word_length = 0
    
    if(typing_word != ''):
        word_length = len(typing_word)
    
    length = len(word)
    words = word[length - 2:length]
    
    suggestion_dic = {}
    
    items = frequency_trigrams.items()
    
    for item in items:
        key = item[0]
        start_word = key[0]
        second_word = key[1]
        last_word = key[2]

        if words[0] == start_word:           
            if words[1] == second_word:
                if word_length == 0:
                    suggestion_dic[last_word] = item[1]
                else:
                    if typing_word == last_word[0:word_length] and word_length <= len(last_word):
                        suggestion_dic[last_word] = item[1]
    
    suggestion_list_temp = sorted(suggestion_dic.items(), key=lambda item: (-item[1], item[0]))
    length = len(suggestion_list_temp)
    
    suggestion_list_temp = suggestion_list_temp[0:cand_nbr] 
    print(suggestion_list_temp)
    
    suggestion_list = []
    
    for tupel in suggestion_list_temp:
        suggestion_list.append(tupel[0])

    return(suggestion_list)                    

In [358]:
next_word_predictions = suggestion_words_tri(tokens)  
next_word_predictions

[('stor', 34), ('liten', 26), ('gammal', 19), ('god', 17), ('sådan', 17)]


['stor', 'liten', 'gammal', 'god', 'sådan']

Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [359]:
current_text = "Det var en g".lower()

In [400]:
# Write your code here
tokens_2 = tokenize(current_text)
current_word_predictions_2 = suggestion_words_tri(tokens_2[0:3], tokens_2[3])

[('gammal', 19), ('god', 17), ('gång', 10), ('ganska', 3), ('glädje', 2)]


In [401]:
current_word_predictions_2

['gammal', 'god', 'gång', 'ganska', 'glädje']

## Checked answers

The system will check these answers: `(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)`

In [402]:
(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)

(423,
 72,
 ['det', 'de', 'den', 'detta', 'denna'],
 ['stor', 'liten', 'gammal', 'god', 'sådan'],
 ['gammal', 'god', 'gång', 'ganska', 'glädje'])

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [403]:
STIL_ID = ["vi1146ol-s"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "22-language_models.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the perplexities and predictions.

In [404]:
ANSWER = str((perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2))
ANSWER

"(423, 72, ['det', 'de', 'den', 'detta', 'denna'], ['stor', 'liten', 'gammal', 'god', 'sådan'], ['gammal', 'god', 'gång', 'ganska', 'glädje'])"

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [405]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [406]:
ASSIGNMENT = 2
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [407]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
                   verify=True)

# from IPython.display import display, JSON
res.json()

{'msg': None,
 'status': 'correct',
 'signature': '1ed73c4b369dbcbd2b2dce8b3ac17e2fc7f7410c5d2561ae288915246f1b1100c6e3ca7dc6045c48312ddd144a39e7dab31cfc5514994c0713406f9bcee62ffc',
 'submission_id': '5ab686f1-b01e-458b-a687-c8d0139887f7'}

## Reading

<p>As an application of n-grams, execute the Jupyter notebook by Peter Norvig <a
        href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">
    here</a>. Just run all the cells and be sure that you understand the code.
    You will find the data <a href="http://norvig.com/ngrams/">here</a>.</p>
<p>In your report, you will also describe one experiment with a long string of words
    your will create yourself or copy from a text you like. You will remove all the punctuation and
    white spaces from this string. Set this string in lowercase letters.</p>
<p>You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and
    run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. </p>
<p>You will comment the segmentation results you obtain with unigram and bigram models.
</p>